In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset with cluster labels
df = pd.read_csv("../Clustering/clusters_output.csv")


In [10]:
# Load the original dataset
df = pd.read_csv("../Dataset/merged_flight_train_with_holidays.csv")

# Preprocess data
df['Year'] = df['Month (YYYY-MM)'].apply(lambda x: int(x.split('-')[0]))
df['Month'] = df['Month (YYYY-MM)'].apply(lambda x: int(x.split('-')[1]))
df.drop(columns=['Month (YYYY-MM)'], inplace=True)

# One-hot encode 'Country' to predict the country
df_one_hot = pd.get_dummies(df['Country'], prefix='Country', drop_first=True)

# Features: 'Arrivals', 'Departures', 'No of holidays', 'Year', 'Month'
X = df[['Arrivals', 'Departures', 'No of holidays', 'Year', 'Month']]

# Target: 'Country'
y = df['Country']

# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import classification_report, confusion_matrix
y_pred = classifier.predict(X_test)

# Print out classification results
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# # Load the original dataset
# df = pd.read_csv("../Dataset/merged_flight_testing_with_holidays.csv")

# # Preprocess data
# df['Year'] = df['Month (YYYY-MM)'].apply(lambda x: int(x.split('-')[0]))
# df['Month'] = df['Month (YYYY-MM)'].apply(lambda x: int(x.split('-')[1]))
# df.drop(columns=['Month (YYYY-MM)'], inplace=True)

# # One-hot encode 'Country' to predict the country
# df_one_hot = pd.get_dummies(df['Country'], prefix='Country', drop_first=True)

# # Features: 'Arrivals', 'Departures', 'No of holidays', 'Year', 'Month'
# X = df[['Arrivals', 'Departures', 'No of holidays', 'Year', 'Month']]

# # Target: 'Country'
# y = df['Country']

# # Split data into train and test sets
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Choose a classification model and fit it (Random Forest, for example)
# from sklearn.ensemble import RandomForestClassifier
# classifier = RandomForestClassifier(random_state=42)
# classifier.fit(X_train, y_train)

# # Evaluate the model
# from sklearn.metrics import classification_report, confusion_matrix
# y_pred = classifier.predict(X_test)

# # Print out classification results
# print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))

# # sample code for new data
# new_data = pd.DataFrame({
#     'Arrivals': [350000],
#     'Departures': [349500],
#     'No of holidays': [2],
#     'Year': [2025],
#     'Month': [5]
# })
# new_predictions = classifier.predict(new_data)
# print("Predicted country for May 2025:", new_predictions)



                precision    recall  f1-score   support

        France       1.00      1.00      1.00        20
       Germany       1.00      0.81      0.89        21
     Hong Kong       0.56      0.31      0.40        16
     Indonesia       0.77      0.94      0.85        18
         Japan       0.50      0.62      0.56        16
Mainland China       0.50      0.29      0.37        17
      Malaysia       0.68      0.72      0.70        18
   Philippines       0.38      0.35      0.36        23
      Thailand       0.54      0.64      0.58        11
United Kingdom       0.75      1.00      0.86        12
       Vietnam       0.64      0.82      0.72        17

      accuracy                           0.68       189
     macro avg       0.67      0.68      0.66       189
  weighted avg       0.67      0.68      0.66       189

[[20  0  0  0  0  0  0  0  0  0  0]
 [ 0 17  0  0  0  0  0  0  0  4  0]
 [ 0  0  5  0  1  0  0  8  0  0  2]
 [ 0  0  0 17  0  0  1  0  0  0  0]
 [ 0  0  1  0